<a href="https://colab.research.google.com/github/nvndvg/crop_suggestion/blob/master/KNN_PE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
install.packages('performanceEstimation')
install.packages('rattle')
install.packages('Hmisc')
install.packages('caret')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘BBmisc’, ‘parallelMap’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [0]:
# Load required packages from local library into the R session.

library(dplyr)                # Data wrangling, glimpse() and tbl_df().
library(ggplot2)              # Visualise data.
library(rattle)               # normVarNames().
library(readr)                # Efficient reading of CSV data.
library(scales)               # Format comma().
library(tibble)               # Convert row names into a column.
library(stringi)              # String concat operator %s+%.
library(stringr)              # String operations.
library(tidyr)                # Prepare a tidy dataset, gather().
library(magrittr)             # Pipes %>%, %T>% and equals(), extract().
library(reshape2)             # Restructure and aggregate data using just two functions: melt()
library(Hmisc)                # The describe() global summary of a dataset.
library(caret)                # DTI, KNN, create partiton.
library(performanceEstimation)# Infrastructure.

In [9]:
cat('Ingest the dataset.\n\n')

url <- 'https://raw.githubusercontent.com/nvndvg/crop_suggestion/master/crop_bigdata_clustered_clustMixType.csv'

ds <- read_csv(url)

Ingest the dataset.



Parsed with column specification:
cols(
  crops = col_character(),
  nitrogen = col_double(),
  phosphorus = col_double(),
  potassium = col_double(),
  ph = col_double(),
  temperature = col_double(),
  annual_rainfall = col_double(),
  season = col_character(),
  soil_type = col_character(),
  clusters = col_double()
)



In [10]:
cat('A quick view of the contents of the dataset.\n\n')

glimpse(ds)

cat('\n\nBasic size information.\n\n')

dim(ds) %>% comcat()

cat('\n\n# of rows\n\n')

nrow(ds) %>% comcat()

cat('\n\n# of columns\n\n')

ncol(ds) %>% comcat()

A quick view of the contents of the dataset.

Rows: 44,280
Columns: 10
$ crops           <chr> "rice", "rice", "rice", "rice", "rice", "rice", "rice…
$ nitrogen        <dbl> 70, 80, 90, 70, 80, 90, 70, 80, 90, 70, 80, 90, 70, 8…
$ phosphorus      <dbl> 30, 30, 30, 40, 40, 40, 50, 50, 50, 30, 30, 30, 40, 4…
$ potassium       <dbl> 30, 30, 30, 30, 30, 30, 30, 30, 30, 40, 40, 40, 40, 4…
$ ph              <dbl> 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5, 5.5…
$ temperature     <dbl> 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2…
$ annual_rainfall <dbl> 175, 175, 175, 175, 175, 175, 175, 175, 175, 175, 175…
$ season          <chr> "kharif", "kharif", "kharif", "kharif", "kharif", "kh…
$ soil_type       <chr> "clay", "clay", "clay", "clay", "clay", "clay", "clay…
$ clusters        <dbl> 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1…


Basic size information.

44,280 10 


# of rows

44,280 


# of columns

10 


In [5]:
cat('\n\n# Note the available variables.\n\n')

ds %>% 
  names() %T>% 
  print()->
vars

cat('\n\n# Note which variables are numeric.\n\n')

vars %>%
  magrittr::extract(ds, .) %>% 
  sapply(is.numeric) %>% 
  which() %>%
  names() %T>% 
  print()->
numc

cat('\n\n# Convert these variables from character to factor and confirm.\n\n')

ds[fct_vars[-1]] %<>%
  lapply(factor) %>%
  data.frame() %>%
  tbl_df() %T>%
  {sapply(., class) %>% print()}





# Note the available variables.

[1] "crops"           "nitrogen"        "phosphorus"      "potassium"      
[5] "ph"              "temperature"     "annual_rainfall" "season"         
[9] "soil_type"      


# Note which variables are numeric.

[1] "nitrogen"        "phosphorus"      "potassium"       "ph"             
[5] "temperature"     "annual_rainfall"


# Convert these variables from character to factor and confirm.

   season soil_type 
 "factor"  "factor" 


In [6]:
cat('\n\n# Convert season variable from factor to binary and confirm.\n\n')

dmy <- dummyVars(" ~ season", data = ds, fullRank = T)
trsf <- data.frame(predict(dmy, newdata = ds))
print(tail(trsf))



# Convert season variable from factor to binary and confirm.

      season.rabi
44275           1
44276           1
44277           1
44278           1
44279           1
44280           1


In [0]:
cat('\n\n# Convert these variables from character to factor and confirm.\n\n')

dmy <- dummyVars(" ~ soil_type", data = ds)
trsf <- data.frame(predict(dmy, newdata = ds))
print(tail(trsf))



# Convert these variables from character to factor and confirm.

      soil_type.alluvial soil_type.black soil_type.clay soil_type.clay_loam
44275                  0               0              1                   0
44276                  0               0              1                   0
44277                  0               0              1                   0
44278                  0               0              1                   0
44279                  0               0              1                   0
44280                  0               0              1                   0
      soil_type.gravely soil_type.laterite soil_type.lateritic soil_type.loam
44275                 0                  0                   0              0
44276                 0                  0                   0              0
44277                 0                  0                   0              0
44278                 0                  0                   0              0
44279      

In [0]:
names(trsf)

[1] "soil_type.alluvial"        "soil_type.black"          
 [3] "soil_type.clay"            "soil_type.clay_loam"      
 [5] "soil_type.gravely"         "soil_type.laterite"       
 [7] "soil_type.lateritic"       "soil_type.loam"           
 [9] "soil_type.loamy"           "soil_type.red"            
[11] "soil_type.red_laterite"    "soil_type.red_loam"       
[13] "soil_type.red_loamy"       "soil_type.red_yellow"     
[15] "soil_type.sandy"           "soil_type.sandy_clay_loam"
[17] "soil_type.sandy_loam"

In [0]:
lst <- names(trsf)
lst

[1] "soil_type.alluvial"        "soil_type.black"          
 [3] "soil_type.clay"            "soil_type.clay_loam"      
 [5] "soil_type.gravely"         "soil_type.laterite"       
 [7] "soil_type.lateritic"       "soil_type.loam"           
 [9] "soil_type.loamy"           "soil_type.red"            
[11] "soil_type.red_laterite"    "soil_type.red_loam"       
[13] "soil_type.red_loamy"       "soil_type.red_yellow"     
[15] "soil_type.sandy"           "soil_type.sandy_clay_loam"
[17] "soil_type.sandy_loam"

In [0]:
ds$lst[] <- trsf

In [0]:
names(ds)

[1] "crops"           "nitrogen"        "phosphorus"      "potassium"      
 [5] "ph"              "temperature"     "annual_rainfall" "season"         
 [9] "soil_type"       "lst"